# Car by Car examination of time dependent data

## Examples are speed plotted out by day but you could adapt to other things

## Usual Library Imports

In [ ]:
import numpy as np
import datetime
from dateutil import tz
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

%matplotlib inline

## Subroutine to load in from data set 1

**David's ideas**

**To do: do the same for data set 2**

In [ ]:
def car1_loc_data(contents):
    data_select=pd.DataFrame(columns=contents)
    for n in np.arange(10):
        print('n=',n,' of ten')
        data_raw=pd.read_csv('~/data/5-Sanandaji/case_study_dt1.csv',
                nrows=100000,skiprows=range(1,n*100000))
        data_raw = data_raw[contents]
        data_select=pd.concat([data_select,data_raw])
        del data_raw
    return data_select

## Function to compute seconds after data start from a time stamp string

In [ ]:
# Routine takes time string from the data and gives back:
# 1. flag = 0: for success, 
#           1: if the input is not a string, 
#           2: if it is not long enough
# 2. seconds since June 9, 2016
#
def get_time(dxt):
    err_flag = 0
    delta_sec = 0
    
    dt0 = datetime.datetime(2016,6,9) # Measure time since June 9, 2016
    
    if not (type(dxt) is str):
        err_flag = 1
    elif len(dxt)<19:
        err_flag = 2
    else:
        year = int(dxt[0:4])
        month = int(dxt[5:7])
        day = int(dxt[8:10])
        hour = int(dxt[11:13])
        minute = int(dxt[14:16])
        second = int(dxt[17:19])
        dt1 = datetime.datetime(
            year, month, day, hour=hour, minute=minute, second=second)
        delta = dt1-dt0
        delta_sec = delta.total_seconds()
        
    return err_flag, delta_sec

## Function to extract local time hour and day of the week

In [ ]:
# Routine to get the hour minute and second and day of the week 
# adjusted from UTC to EST. Returns:
# 1. hour
# 2. day of the week 0-6 M-Su
# Note: no error checking, assumes past get_time() checks above 
def get_day(dxt,state):
    from_zone = tz.gettz('UTC')
    if state == "Ontario" or state == "Massachusetts" or state == "Michigan" \
        or state == "New Jersey" or state =="New York" or state =="Florida":
        zone = "EST"
    elif state == "Arizona" or state == "Kansas" or state =="Oklahoma" \
        or state =="Utah" or state =="Mountain":
        zone = "MST"
    elif state == "Iowa" or state =="Tennessee":
        zone == "CST"
    elif state == "California" or state == "British Columbia" or state == "Oregon":
        zone = "PST"
    elif state == "UST":
        zone = "UST"
    elif state == "Alaska":
        zone = "PST"
    else:
        print("Time Zone for State", state, " must be added")
        print("Do this before proceeding")
        zone = "UST"
    to_zone = tz.gettz(zone)
    year = int(dxt[0:4])
    month = int(dxt[5:7])
    day = int(dxt[8:10])
    hour = int(dxt[11:13])
    minute = int(dxt[14:16])
    second = int(dxt[17:19])
    dt_utc = datetime.datetime(
        year, month, day, hour=hour, minute=minute, second=second)
    dt_utc = dt_utc.replace(tzinfo=from_zone)
    dt_local = dt_utc.astimezone(to_zone)
    return dt_local.hour, dt_local.minute, dt_local.second, dt_local.weekday()

In [ ]:
# print(time_string[0])
# get_day(time_string[0],'Alaska')

## Load in the data from set 1 from the columns you want 

**Ten lots of size 100,000**

**To do: append the data from set #2**

*Notice that there are missing values in the set with nan values even in string columns*

In [ ]:
# columns to try and identify when vehicles are refueled 
cols_gas = ["source_id", "source_Vehicle_Location_Timestamp", "source_Vehicle_Speed_Unit", 
            "source_Vehicle_Speed_Value", "source_Vehicle_FuelLevel_Value", 
            "source_Vehicle_Location_Address_State", "source_EventCode"]
data_gas = car1_loc_data(cols_gas)
list(data_gas)

## Read in the list of cars from set #1

**To do: add the cars from set #2**

In [ ]:
sources = pd.read_csv("~/bcdata-mojio/brian/test1_sources.txt")
sources = sources["Car IDs"].values
print('Number of cars:',sources.size)

# **Pick a car!**

Get all the records for that car

*Cars # 2, 3, 8, are interesting*



In [ ]:
car_number = 8
car_info = data_gas[data_gas["source_id"]==sources[car_number]]
Ncar = car_info.shape[0] # number of records for this car

**Get the states the car has visited. Fill in nan values**

In [ ]:
state = car_info["source_Vehicle_Location_Address_State"].values
for j in np.arange(Ncar):
    if state[j] is np.nan:
        state[j] = "UTC"
states = np.unique(state)
flags = states=="UTC"
if np.sum(flags) > 0 and states.size > 1:
        print('Replacing UTC (nan in data) with known state') 
        state[state=="UTC"] = states[states != "UTC"][0]
states = np.unique(state)
print('States for this car:', states)

**See if the speeds are mph**

In [ ]:
speed_units = car_info["source_Vehicle_Speed_Unit"]
speed_unit = np.unique(speed_units.values)
speed_conversion_tokph = 1
if speed_unit[0] == 'MilesPerHour':
    speed_conversion_tokph = 1.61
    print('Converting to kph')

**Compute the times of the records**

*Note that the data is not sorted by time*

In [ ]:
time_string = car_info["source_Vehicle_Location_Timestamp"].values
time_seconds = np.zeros(Ncar)
for j in np.arange(Ncar):
    code, t = get_time(time_string[j])
    if code != 0:
        print('Error',code,' in get_time')
    time_seconds[j] = t

## Sort the car data by time

In [ ]:
car_info.insert(1,"seconds",time_seconds) # don't execute this twice for the same car

In [ ]:
car_info2 = car_info.sort_values("seconds")
time_sorted = car_info2["seconds"].values

## Now look at the time history for this car

**Speed versus time**

*Car #2 is a fast driver*

In [ ]:
speed = car_info2["source_Vehicle_Speed_Value"].values
plt.plot(time_sorted/24/3600,speed*speed_conversion_tokph)
plt.ylabel("Speed kph")
plt.xlabel("Time in days past June 9, 2016")
plt.show()

### Let us see the driver's daily driving habits

In [ ]:
time_string = car_info2["source_Vehicle_Location_Timestamp"].values
time_hour = np.zeros(Ncar)
time_minute = np.zeros(Ncar)
time_second = np.zeros(Ncar)
time_dayofweek = np.zeros(Ncar)
for j in np.arange(Ncar):
    hour, minute, second, day = get_day(time_string[j],state[j])
    time_hour[j] = hour
    time_dayofweek[j] = day
    time_minute[j] = minute
    time_second[j] = second
plt.figure(figsize=(10,6))
plt.xlabel('Hour of the day (local time)')
plt.ylabel('Speed kph')
plt.plot(time_hour+time_minute/60+time_second/3600,speed*speed_conversion_tokph, 'ro')
plt.show()

## Let us separate out weekdays from weekends to see if we see more of a pattern

*Car # 3 commutes either early or late*

*Car # 8 sleeps all weekend*

In [ ]:
times = time_hour+time_minute/60+time_second/3600
speeds = speed*speed_conversion_tokph

times1 = times[time_dayofweek >4] # weekends
speeds1 = speeds[time_dayofweek >4]
times2 = times[time_dayofweek < 5]
speeds2 = speeds[time_dayofweek < 5]

fig = plt.figure(figsize=(18,6))
ax1=fig.add_subplot(1,2,1)
ax1.plot(times1, speeds1, 'go')
ax1.set_xlabel('Hour of the Day')
ax1.set_ylabel('Speed kph')
ax1.set_title('Weekends')

ax2=fig.add_subplot(1,2,2)
ax2.plot(times2, speeds2, 'ro')
ax2.set_xlabel('Hour of the Day')
ax2.set_ylabel('Speed kph')
ax2.set_title('Weekdays')

plt.show()


Separate Sunday and Friday

*Looks like Car # 3 might go to church on Sunday? Or golfing/the gym/?*

In [ ]:
times1 = times[time_dayofweek == 6] # Sunday
speeds1 = speeds[time_dayofweek == 6]
times2 = times[time_dayofweek == 4] #Friday
speeds2 = speeds[time_dayofweek == 4]

fig = plt.figure(figsize=(18,6))
ax1=fig.add_subplot(1,2,1)
ax1.plot(times1, speeds1, 'go')
ax1.set_xlabel('Hour of the Day')
ax1.set_ylabel('Speed kph')
ax1.set_title('Sundays')

ax2=fig.add_subplot(1,2,2)
ax2.plot(times2, speeds2, 'ro')
ax2.set_xlabel('Hour of the Day')
ax2.set_ylabel('Speed kph')
ax2.set_title('Fridays')

plt.show()

## Plot the data day by day 

**Avoid some difficulty here because we only have one week of data**

In [ ]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
fig = plt.figure(figsize=(16,32))
for day in np.arange(7):
    ax=fig.add_subplot(4,2,day+1)
    day_times = times[time_dayofweek == day]
    day_speeds = speeds[time_dayofweek == day]
    ax.plot(day_times, day_speeds, 'bo')
    ax.set_xlabel("Hour of the day")
    ax.set_ylabel("Speed kph")
    ax.set_title(days[day])
plt.show()